<a href="https://colab.research.google.com/github/Tsung-Hung/dummy-git/blob/master/%E6%AD%A1%E8%BF%8E%E4%BD%BF%E7%94%A8_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch 
import torch.nn as nn

# LayerNorm replace by BatchNorm

In [2]:
batch, sentence_length, embedding_dim = 10, 3, 5
embedding = torch.randn(batch, sentence_length, embedding_dim)
layer_norm = nn.LayerNorm(embedding_dim)
# Activate module
layer_norm(embedding)

tensor([[[-1.7308,  0.3394, -0.3598,  1.2118,  0.5395],
         [-1.6366, -0.0361,  1.0493, -0.4040,  1.0275],
         [ 1.1531,  1.0474, -0.2123, -0.4690, -1.5192]],

        [[-1.1769,  0.8899, -0.7252, -0.4385,  1.4507],
         [-1.1468,  0.9449, -0.9675, -0.1808,  1.3502],
         [-0.4503,  0.3332, -1.3766, -0.1684,  1.6621]],

        [[-0.4306, -0.6724, -0.4838,  1.9912, -0.4045],
         [ 0.7504,  0.5666, -1.2971, -1.1130,  1.0930],
         [ 0.7756, -0.0152,  0.6368, -1.9267,  0.5295]],

        [[ 1.1373,  0.5529,  0.7118, -1.1326, -1.2694],
         [ 1.2813, -0.2185, -0.9963,  1.0428, -1.1092],
         [ 0.1431,  1.7945, -0.5393, -1.1943, -0.2041]],

        [[-0.8677, -0.4115,  1.5856,  0.7176, -1.0240],
         [ 0.5900,  0.5884,  0.8392, -1.8937, -0.1239],
         [-1.2823, -0.2440,  0.0508, -0.3122,  1.7877]],

        [[-1.4657, -0.6722,  0.4679,  1.4621,  0.2079],
         [ 0.1624,  0.7439,  0.2070, -1.9273,  0.8140],
         [-1.3112,  1.0268, -0.3842, -

In [3]:
def Layer_norm(x):
    batch, sentence_length, embedding_dim = x.shape
    x = x.reshape(1, batch*sentence_length, embedding_dim, 1)
    print(x.shape)
    layer_norm = nn.BatchNorm2d(x.shape[1], track_running_stats=False, affine=True)
    output = layer_norm(x)
    output = output.reshape(output.shape[1], output.shape[2]).reshape(batch, sentence_length, embedding_dim)
    return output

test = Layer_norm(embedding)
test

torch.Size([1, 30, 5, 1])


tensor([[[-1.7308,  0.3394, -0.3598,  1.2118,  0.5395],
         [-1.6366, -0.0361,  1.0493, -0.4040,  1.0275],
         [ 1.1531,  1.0474, -0.2123, -0.4690, -1.5192]],

        [[-1.1769,  0.8899, -0.7252, -0.4385,  1.4507],
         [-1.1468,  0.9449, -0.9675, -0.1808,  1.3502],
         [-0.4503,  0.3332, -1.3766, -0.1684,  1.6621]],

        [[-0.4306, -0.6724, -0.4838,  1.9912, -0.4045],
         [ 0.7504,  0.5666, -1.2971, -1.1130,  1.0930],
         [ 0.7756, -0.0152,  0.6368, -1.9267,  0.5295]],

        [[ 1.1373,  0.5529,  0.7118, -1.1326, -1.2694],
         [ 1.2813, -0.2185, -0.9963,  1.0428, -1.1092],
         [ 0.1431,  1.7945, -0.5393, -1.1943, -0.2041]],

        [[-0.8677, -0.4115,  1.5856,  0.7176, -1.0240],
         [ 0.5900,  0.5884,  0.8392, -1.8937, -0.1239],
         [-1.2823, -0.2440,  0.0508, -0.3122,  1.7877]],

        [[-1.4657, -0.6722,  0.4679,  1.4621,  0.2079],
         [ 0.1624,  0.7439,  0.2070, -1.9273,  0.8140],
         [-1.3112,  1.0268, -0.3842, -

# matmul 4D tensor

In [4]:
a1 = torch.rand(2,2,4,8)
a2 = torch.rand(2,2,6,8)

result = torch.matmul(a1, a2.transpose(-2,-1))
print('Shape : ', result.shape)
result

Shape :  torch.Size([2, 2, 4, 6])


tensor([[[[2.8331, 2.7717, 1.8197, 1.9072, 2.6365, 2.8051],
          [2.2667, 2.0982, 1.5836, 1.6143, 2.1525, 2.1333],
          [1.9435, 1.5990, 1.3051, 1.4194, 1.3878, 1.6595],
          [3.0025, 3.4030, 2.4411, 2.8075, 2.6819, 3.4550]],

         [[2.9619, 1.8788, 2.7696, 2.1046, 2.2509, 2.8159],
          [2.8309, 1.9611, 2.5310, 2.0730, 2.3487, 2.7794],
          [2.6447, 2.1122, 2.8377, 2.2849, 2.1042, 3.0009],
          [2.5675, 1.7580, 2.6901, 1.9674, 1.8271, 2.6649]]],


        [[[2.2773, 2.0813, 3.1141, 2.5992, 2.3233, 4.1886],
          [2.2283, 1.8513, 2.5137, 2.5783, 2.4522, 3.2121],
          [1.7229, 2.0190, 2.2074, 2.5631, 2.3288, 3.2346],
          [2.1737, 2.2139, 2.5276, 2.6603, 2.8101, 3.9480]],

         [[1.3014, 3.3570, 2.5664, 3.0435, 2.9156, 1.9071],
          [0.8277, 2.2444, 1.6118, 2.1012, 1.9516, 1.4173],
          [1.6606, 3.1645, 2.2176, 2.9234, 2.7169, 1.7726],
          [1.6313, 2.6954, 1.8212, 2.1942, 2.7870, 1.5543]]]])

In [5]:
def new_matmul1(a1,a2):

    k = a1.shape[0]*a1.shape[1]    
    mul_list = []
    for i in range(a1.shape[0]):
        for j in range(a1.shape[1]):
            
            mul = torch.matmul(a1[i,j,:],a2[i,j,:])                
            mul_list.append(mul)

    combine = torch.stack(mul_list)  
    combine = combine.view(a1.shape[0],a1.shape[1],combine.shape[1],combine.shape[2])
    
    return combine

result1 = new_matmul1(a1, a2.transpose(-2,-1))
print('Shape : ', result1.shape)
print(result1)

Shape :  torch.Size([2, 2, 4, 6])
tensor([[[[2.8331, 2.7717, 1.8197, 1.9072, 2.6365, 2.8051],
          [2.2667, 2.0982, 1.5836, 1.6143, 2.1525, 2.1333],
          [1.9435, 1.5990, 1.3051, 1.4194, 1.3878, 1.6595],
          [3.0025, 3.4030, 2.4411, 2.8075, 2.6819, 3.4550]],

         [[2.9619, 1.8788, 2.7696, 2.1046, 2.2509, 2.8159],
          [2.8309, 1.9611, 2.5310, 2.0730, 2.3487, 2.7794],
          [2.6447, 2.1122, 2.8377, 2.2849, 2.1042, 3.0009],
          [2.5675, 1.7580, 2.6901, 1.9674, 1.8271, 2.6649]]],


        [[[2.2773, 2.0813, 3.1141, 2.5992, 2.3233, 4.1886],
          [2.2283, 1.8513, 2.5137, 2.5783, 2.4522, 3.2121],
          [1.7229, 2.0190, 2.2074, 2.5631, 2.3288, 3.2346],
          [2.1737, 2.2139, 2.5276, 2.6603, 2.8101, 3.9480]],

         [[1.3014, 3.3570, 2.5664, 3.0435, 2.9156, 1.9071],
          [0.8277, 2.2444, 1.6118, 2.1012, 1.9516, 1.4173],
          [1.6606, 3.1645, 2.2176, 2.9234, 2.7169, 1.7726],
          [1.6313, 2.6954, 1.8212, 2.1942, 2.7870, 1.5543]

In [6]:
def new_matmul2(a1, a2):

    combine = torch.stack([torch.matmul(a1[i][j], a2[i][j]) for i in range(a1.shape[0]) for j in range(a1.shape[1])], dim=0)        
    return combine.reshape(a1.shape[0], a1.shape[1], a1.shape[2], a2.shape[3])

result2 = new_matmul1(a1, a2.transpose(-2,-1))
print('Shape : ', result2.shape)
print(result2)
print(torch.all(result==result2))

Shape :  torch.Size([2, 2, 4, 6])
tensor([[[[2.8331, 2.7717, 1.8197, 1.9072, 2.6365, 2.8051],
          [2.2667, 2.0982, 1.5836, 1.6143, 2.1525, 2.1333],
          [1.9435, 1.5990, 1.3051, 1.4194, 1.3878, 1.6595],
          [3.0025, 3.4030, 2.4411, 2.8075, 2.6819, 3.4550]],

         [[2.9619, 1.8788, 2.7696, 2.1046, 2.2509, 2.8159],
          [2.8309, 1.9611, 2.5310, 2.0730, 2.3487, 2.7794],
          [2.6447, 2.1122, 2.8377, 2.2849, 2.1042, 3.0009],
          [2.5675, 1.7580, 2.6901, 1.9674, 1.8271, 2.6649]]],


        [[[2.2773, 2.0813, 3.1141, 2.5992, 2.3233, 4.1886],
          [2.2283, 1.8513, 2.5137, 2.5783, 2.4522, 3.2121],
          [1.7229, 2.0190, 2.2074, 2.5631, 2.3288, 3.2346],
          [2.1737, 2.2139, 2.5276, 2.6603, 2.8101, 3.9480]],

         [[1.3014, 3.3570, 2.5664, 3.0435, 2.9156, 1.9071],
          [0.8277, 2.2444, 1.6118, 2.1012, 1.9516, 1.4173],
          [1.6606, 3.1645, 2.2176, 2.9234, 2.7169, 1.7726],
          [1.6313, 2.6954, 1.8212, 2.1942, 2.7870, 1.5543]

In [7]:
def new_matmul3(a1, a2):
  '''  
  a1 = (2,2,4,8)  
  a2 = (2,2,6,8) 
  torch(a2, a1.T)
  '''
  B1, N1, H1, T1 = a1.shape 
  B2, N2, H2, T2 = a2.shape 
  a1 = a1.reshape(B1 * N1 * H1, T1)
  #print('a1 : ', a1.shape)
  #a2 = a2.transpose(-2,-1)
  a2 = a2.permute(2,0,1,3)
  a2 = a2.reshape(T2, B2 * N2 * H2)
  #print('a2 : ', a2.shape)
  result = torch.matmul(a1, a2) 
  result = torch.stack([result[i*H1:(i+1)*H1 ,i*H2:(i+1)*H2] for i in range(B1*N1)])
  result = result.reshape(B1,N1,H1,H2)
  return result

result3 = new_matmul3(a1, a2)
print('Shape : ', result3.shape)
print(result3)

Shape :  torch.Size([2, 2, 4, 6])
tensor([[[[1.2786, 2.2635, 1.5673, 2.9284, 2.5029, 2.0595],
          [1.2368, 1.8651, 1.6052, 2.8708, 1.6599, 1.8963],
          [0.8444, 1.1626, 1.2507, 1.5021, 1.8573, 1.2675],
          [1.6751, 3.3484, 2.5815, 3.0021, 3.4962, 3.0348]],

         [[1.4606, 2.9339, 2.3312, 2.2957, 3.1507, 2.9177],
          [1.3388, 2.5807, 2.2758, 2.7728, 2.9638, 2.9332],
          [1.3355, 2.7264, 2.3212, 2.7953, 3.0770, 2.8367],
          [1.4994, 3.0412, 1.8695, 2.5386, 2.8736, 2.3483]]],


        [[[3.5235, 3.1677, 4.0275, 3.6679, 2.9562, 3.2868],
          [2.6863, 2.4466, 3.2364, 3.4208, 2.6917, 2.1805],
          [2.7143, 2.2988, 3.6185, 3.2199, 2.6388, 2.2886],
          [3.4260, 2.6700, 3.4653, 2.9030, 2.7746, 2.7606]],

         [[2.1604, 2.3168, 2.1566, 1.5830, 2.5715, 2.3203],
          [1.5586, 1.3905, 1.4417, 1.1030, 1.3046, 1.8581],
          [2.5540, 1.7719, 1.9505, 1.4387, 1.9333, 2.3759],
          [2.3719, 1.4620, 1.9287, 1.0728, 2.2228, 1.7634]

In [69]:
def new_matmul4(a1, a2):
  '''
  a1 = (2,2,4,8)  
  a2 = (2,2,6,8)
  a1_matrix = (16,32)
  a2_matrix = (32,24)
  '''
  a1_matrix = torch.block_diag(*[a1[i][j] for i in range(a1.shape[0]) for j in range(a1.shape[1])])
  a2_matrix = torch.block_diag(*[a2[i][j] for i in range(a2.shape[0]) for j in range(a2.shape[1])])
  #identity = torch.cat([torch.eye(a2.shape[3]) for _ in range(a2.shape[0]*a2.shape[1])]).cuda()
  a = torch.ones(a2.shape[0]*a2.shape[1],a2.shape[3])
  #identity2 = torch.diag_embed(a).reshape(a2.shape[0]*a2.shape[1]*a2.shape[3], a2.shape[3]).cuda()
  result = torch.matmul(a1_matrix, a2_matrix)
  #result = torch.matmul(result, identity2)
  return result

#result4 = new_matmul4(a1,a2.transpose(-2,-1))
#print('Shape : ', result4.shape)
#print(result4)

In [61]:
a1 = torch.rand(1,4,15000,16).cuda()
a2 = torch.rand(1,4,1000,16).cuda()

%timeit -n 100 -r 7 torch.matmul(a1, a2.transpose(-2,-1))

23.3 µs ± 3.68 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [62]:
%timeit -n 100 -r 7 new_matmul1(a1, a2.transpose(-2,-1))

The slowest run took 23.61 times longer than the fastest. This could mean that an intermediate result is being cached.
3.24 ms ± 1.28 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [63]:
%timeit -n 100 -r 7 new_matmul2(a1, a2.transpose(-2,-1))

3.58 ms ± 47.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [64]:
#%timeit -n 200 -r 7 new_matmul3(a1, a2)

In [70]:
%timeit -n 100 -r 7 new_matmul4(a1, a2.transpose(-2,-1))

The slowest run took 8.82 times longer than the fastest. This could mean that an intermediate result is being cached.
6.67 ms ± 2.35 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [66]:
a1_matrix = torch.block_diag(*[a1[i][j] for i in range(a1.shape[0]) for j in range(a1.shape[1])])
a2_matrix = torch.block_diag(*[a2[i][j] for i in range(a2.shape[0]) for j in range(a2.shape[1])])

%timeit -n 100 -r 7 torch.matmul(a1_matrix, a2_matrix.transpose(-2,-1))

50.7 µs ± 11.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
